In [ ]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os
from pebble import lattice

In [ ]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

## Data Loading

### Writing PyG Dataset

In [3]:
import torch

In [4]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [5]:
from torch_geometric.utils import from_networkx, to_networkx

In [6]:
def clustering_coefficient(G, node):
    ns = [n for n in G.neighbors(node)]
    if len(ns) <= 1:
        return 0
    
    numerator = 0
    denominator = len(ns) * (len(ns) - 1) / 2
    for i in range(0, len(ns)):
        for j in range(i+1, len(ns)):
            n1, n2 = ns[i], ns[j]
            numerator += G.has_edge(n1, n2)
    
    return numerator / denominator
                

In [7]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 4)
    ind = 0
    for node in G.nodes():
        x[ind][0] = 1 # uniform
        x[ind][1] = G.degree[node] # node degree as a scalar 
        x[ind][2] = clustering_coefficient(G, node) # triangle counting?
        x[ind][3] = ind # node ID features
        ind += 1
    return x

In [8]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        for ind, graph in enumerate(total_laman_data[0]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        for ind, graph in enumerate(total_laman_data[1]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [9]:
DATA_PATH = "data/incorrectly-predicted.pkl.gz"

In [10]:
laman_data = LamanDataset("", DATA_PATH)

In [11]:
laman_data[0]

Data(edge_index=[2, 64], x=[15, 4], label=[1], num_nodes=15)

## Split into Train / Test

In [12]:
from torch.utils.data import random_split

proportions = [.7, .3]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [13]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 256, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 256, shuffle=True)

In [14]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  27
Number of test batches:  12


In [15]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 14974], x=[3840, 4], label=[256], num_nodes=3840, batch=[3840], ptr=[257])


## Model Architecture & Training

In [16]:
from basic_gcn.gcn import GCN

In [21]:
model = GCN(num_features=4, embedding_size=15)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(4, 15)
  (conv1): GCNConv(15, 15)
  (out): Linear(in_features=60, out_features=60, bias=True)
  (out2): Linear(in_features=60, out_features=1, bias=True)
)
Number of parameters:  4036


In [22]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

# scheduler = ReduceLROnPlateau(optimizer, 'min', min_lr=1e-6, verbose=True, patience=10)

In [23]:
def train(data, features_to_use):
    ind = 0
    model.train()
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, None

In [24]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [25]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(100):
    loss, _ = train(train_loader, [0, 1, 2, 3])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader, [0, 1, 2, 3]), check_accuracy(model, test_loader, [0, 1, 2, 3])
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...
Epoch 0 | Train loss 0.6999048590660095
Train Accuracy 55.693950177935946 | Test Accuracy 55.51712210307852
Epoch 1 | Train loss 0.7237244248390198
Train Accuracy 55.693950177935946 | Test Accuracy 55.51712210307852
Epoch 2 | Train loss 0.6852387189865112
Train Accuracy 57.07295373665481 | Test Accuracy 56.10515392597717
Epoch 3 | Train loss 0.6905332207679749
Train Accuracy 57.443653618030844 | Test Accuracy 56.10515392597717
Epoch 4 | Train loss 0.689138650894165
Train Accuracy 56.16844602609727 | Test Accuracy 55.932203389830505
Epoch 5 | Train loss 0.6662771105766296
Train Accuracy 56.791221826809014 | Test Accuracy 56.3472846765825
Epoch 6 | Train loss 0.6785815358161926
Train Accuracy 59.20818505338078 | Test Accuracy 56.416464891041166
Epoch 7 | Train loss 0.6938939690589905
Train Accuracy 60.5723606168446 | Test Accuracy 57.59252853683846
Epoch 8 | Train loss 0.6713453531265259
Train Accuracy 59.29715302491103 | Test Accuracy 57.938429609131795
Epoch 9 | Tr

Epoch 76 | Train loss 0.39618417620658875
Train Accuracy 80.99051008303677 | Test Accuracy 80.66413005880318
Epoch 77 | Train loss 0.5435583591461182
Train Accuracy 79.49288256227757 | Test Accuracy 79.00380491179523
Epoch 78 | Train loss 0.4749833941459656
Train Accuracy 79.86358244365361 | Test Accuracy 79.31511587685922
Epoch 79 | Train loss 0.45353591442108154
Train Accuracy 81.00533807829181 | Test Accuracy 80.97544102386718
Epoch 80 | Train loss 0.4578724801540375
Train Accuracy 80.66429418742585 | Test Accuracy 80.07609823590454
Epoch 81 | Train loss 0.5014990568161011
Train Accuracy 72.84994068801898 | Test Accuracy 72.15496368038741
Epoch 82 | Train loss 0.4941251873970032
Train Accuracy 80.20462633451957 | Test Accuracy 80.00691802144587
Epoch 83 | Train loss 0.5103566646575928
Train Accuracy 80.99051008303677 | Test Accuracy 80.83708059494984
Epoch 84 | Train loss 0.43231886625289917
Train Accuracy 80.84223013048636 | Test Accuracy 81.04462123832585
Epoch 85 | Train loss 0.4

In [ ]:
# sanity check

In [ ]:
# test on best model

In [ ]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [ ]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)

In [ ]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [ ]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [ ]:
# no triangle and rigid
rigid = nx.Graph()
rigid.add_edge(0, 1)
rigid.add_edge(0, 2)
rigid.add_edge(0, 4)
rigid.add_edge(1, 2)
rigid.add_edge(1, 5)
rigid.add_edge(2, 3)
rigid.add_edge(3, 4)
rigid.add_edge(3, 5)
rigid.add_edge(4, 5)

In [ ]:
model.eval()
bestModel.eval()

In [ ]:
toy_problems = [square, square_bar, triangle, pentagon, rigid]
labels = [1, 0, 0, 1, 0]

for index, toy_problem in enumerate(toy_problems):
    graph_as_data = from_networkx(toy_problem)
    graph_as_data.x = generate_feature_vector(toy_problem)
#     print(generate_feature_vector(toy_problem))
#     graph_as_data.label = labels[index]
    validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
    for batch in validation_set:
        model.eval()
        with torch.no_grad():
#             print(batch.x[:, [0, 1, 2, 3]])
#             print("******")
#             print(batch.edge_index)
#             print("******")
#             print(batch.batch)
#             print("******")
            pred = model(batch.x[:, [0, 1, 2, 3]], batch.edge_index, batch.batch)
            print(pred)
#     break
    

In [ ]:
# generate the bad examples

In [ ]:
test_to_generate_bad_data = DataLoader(test_data, batch_size = 1, shuffle=True)

In [ ]:
train_to_generate_bad_data = DataLoader(train_data, batch_size = 1, shuffle=True)

In [ ]:
print(len(test_to_generate_bad_data))

In [ ]:
check_accuracy(model, train_to_generate_bad_data, [0, 1, 2, 3])

In [ ]:
incorrectly_predicted_flexible_graphs = []

In [ ]:
incorrectly_predicted_rigid_graphs = []

In [ ]:
print(len(incorrectly_predicted_flexible_graphs))

In [ ]:
print(len(incorrectly_predicted_rigid_graphs))

In [ ]:
check_accuracy(model, test_to_generate_bad_data, [0, 1, 2, 3])

In [ ]:
output_file = "incorrectly-predicted.pkl.gz"
with gzip.open(output_file, 'wb') as f:
    pickle.dump((incorrectly_predicted_rigid_graphs, incorrectly_predicted_flexible_graphs), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
print(len(train_to_generate_bad_data))

In [ ]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            if not (predictions == y).sum():
                graph = to_networkx(batch, to_undirected = True)
                if y[0] == 0:
                    incorrectly_predicted_rigid_graphs.append(graph)
                elif y[0] == 1:
                    incorrectly_predicted_flexible_graphs.append(graph)
                
            num_samples += 1
            
    return float(num_correct)/float(num_samples)*100

In [ ]:
print(model.training)
model.train()
print(model.training)

In [ ]:
rigid_data, not_rigid_data = [], []
stats = {}
stats_considered = {}
prev_graphs = []

stats_wrong = {}

num_nodes = 30
for p in np.arange(0.01, 0.3, 0.01):
    stats[p] = 0
    stats_wrong[p] = 0
    for num_graphs in range(1000):
        G = nx.erdos_renyi_graph(num_nodes, p)        
        l = lattice()
        num_edges = 0

        for (u, v) in G.edges():
            if l.add_bond(u, v):
                num_edges += 1

        label = 1
        rigid = False
        if num_edges >= (num_nodes * 2) - 3: # rigid 
            rigid = True
            stats[p] += 1
            label = 0

        graph_as_data = from_networkx(G)
        graph_as_data.x = generate_feature_vector(G)
        validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
        for batch in validation_set:
            pred = model(batch.x[:, [0]], batch.edge_index, batch.batch)
            pred_label = 1
            if (pred[0][0][0] < 0.5):
                pred_label = 0
                
            if pred_label != label:
                print(pred[0][0][0] , " ", label)
                stats_wrong[p] += 1
                print("wrong: , with number of edges: " , G.number_of_edges(), " ", num_edges)
                
    print(stats[p])

In [ ]:
print(stats)

In [ ]:
print(stats_wrong)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(list(stats_wrong.keys()), stats_wrong.values(), color='g')
plt.show()

In [ ]:
(sum(stats_wrong.values())) / (len(stats_wrong) * 1000)

In [ ]:
len(stats_wrong) * 10000

In [ ]:
graph_as_data

In [ ]:
rigid_data, not_rigid_data = [], []
stats = {}
prev_graphs = []

num_nodes = 30
for p in np.arange(0.01, 0.3, 0.01):
    stats[p] = 0
    for num_graphs in range(1000):
        G = nx.erdos_renyi_graph(num_nodes, p)
        l = lattice()
        num_edges = 0

        for (u, v) in G.edges():
            if l.add_bond(u, v):
                num_edges += 1

        rigid = False
        if num_edges >= (num_nodes * 2) - 3: # rigid 
            rigid_data.append(G)
            stats[p] += 1
        else:
            not_rigid_data.append(G)

        prev_graphs.append(G)
        
    print(stats[p])

In [ ]:
rigid_data_1_wrong, not_rigid_data_1_wrong = [], []
stats_wrong = {}
stats_wrong_cum = {}
prev_graphs = []

num_nodes = 30
for num_edges in range(57, 200):
    model.eval()
    stats_wrong[num_edges] = 0
    for num_graphs in range(10):
        G = generate_rigid_nodes_edges(num_nodes, num_edges)
        
#         G = nx.erdos_renyi_graph(num_nodes, p)
#         generate_rigid_nodes_edges()
        graph_as_data = from_networkx(G)
        graph_as_data.x = generate_feature_vector(G)
#         graph_as_data.label = labels[index]
        validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
        for batch in validation_set:
            with torch.no_grad():
                pred = model(batch.x[:, [0, 1, 2, 3]], batch.edge_index, batch.batch)
                if (pred[0][0][0] > 0.5):
                    stats_wrong[num_edges] += 1
                    print("WRONG")
                

print(stats_wrong)

In [ ]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [ ]:
for batch in validation_set:
    pred = bestModel(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

In [ ]:
class LamanTestDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_test.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
# add functionality to support a test dataset
TEST_DATA_PATH = "../data-2d/data/test-dataset-30loc-5std.pkl.gz"
laman_test_set = LamanTestDataset("", TEST_DATA_PATH)

In [ ]:
from torch_geometric.loader import DataLoader


In [ ]:
from torch_geometric.loader import DataLoader
laman_test_loader = DataLoader(laman_test_set, batch_size = 2, shuffle=True)

In [ ]:
random_test_acc = check_accuracy(model, test_loader, [0, 1, 2, 3])
print(f"Accuracy {random_test_acc}")

In [ ]:
# generate statistics on the data

In [ ]:
# test the clustering coefficient

In [ ]:
clustering_coefficient(square, 0)

In [ ]:
clustering_coefficient(triangle, 0)

In [ ]:
clustering_coefficient(square_bar, 0)

In [ ]:
# generate graph correlating clustering coefficient to rigidity

In [ ]:
for item in train_data:
    item = to_networkx(item)
    print(type(item))
    
    break

In [ ]:
torch_geometric.utils.convert.to_networkx()

# Scratch Work: Sahil

In [ ]:
total_laman_data = None
with gzip.open(DATA_PATH, 'r') as f:
    total_laman_data = pickle.load(f)

In [ ]:
sample_graph = total_laman_data[0][0]

In [ ]:
print(type(sample_graph))

In [ ]:
to_data = from_networkx(sample_graph)
from_data = to_networkx(to_data, to_undirected = True)

In [ ]:
print(type(from_data))

In [ ]:
def compute_min_clustering_coefficient(G):
    min_coefficient = 1
    for node in G.nodes():
        min_coefficient = min(min_coefficient, clustering_coefficient(G, node))
        
    return min_coefficient

In [ ]:
for index, sample_graph in enumerate(train_data):
    label = sample_graph.label
    networkx_sample_graph = to_networkx(sample_graph, to_undirected = True)
    print(label, " ", index, " ", compute_min_clustering_coefficient(networkx_sample_graph))
    
    if index == 10:
        break

In [ ]:
print(compute_min_clustering_coefficient(from_data))

In [ ]:
compute_min_clustering_coefficient(triangle)

In [ ]:
model_just_degree = GIN(num_features=1)
print(model_just_degree)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, h = train(train_loader, [0, 1])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model_just_degree, train_loader), check_accuracy(model_just_degree, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

In [ ]:
generate_feature_vector(sample_graph)

In [ ]:
sample_graph.x[:, [0, 1]]

In [ ]:
import importlib

In [ ]:
from data_gen import generate_rigid_nodes_edges

In [ ]:
import sys
sys.path.insert(0, '/Users/sahiljain/Documents/Fall 2022/Independent Work/reversible-inductive-construction/code/genric/laman')

In [ ]:
clustering_coefficient(triangle, 0)

In [ ]:
# save the torch model

In [ ]:
torch.save(model.state_dict(), f"gcn-model-filtered-data.pt")